# Validación de Integración: Coherencia INE ↔ EUROSTAT

**Nombre del archivo:** 02c_validacion_integracion.ipynb
**Objetivo:** Validar la coherencia y compatibilidad entre las tablas INE y EUROSTAT para asegurar que los datos pueden integrarse correctamente.
**Fuentes de datos:**
- INE: Encuesta de Condiciones de Vida (ECV), tablas INE_AROPE_Hogar, INE_Gini_S80S20_CCAA
- EUROSTAT: Tablas EUROSTAT_AROP_Espana, EUROSTAT_Gini_Espana, EUROSTAT_S80S20_Espana
**Fecha de última edición:** 2025-11-16
**Autor:** Mario (databamario)

## Arquitectura Modular
- **Prerequisitos:** `02a_validacion_INE.ipynb` y `02b_validacion_EUROSTAT.ipynb` completados
- **Este módulo:** Validación de coherencia cruzada entre fuentes
- **Orquestador:** `02_run_validation.py` (ejecuta todos los módulos)

## Validaciones de Integración
1. **Coherencia temporal:** Años compatibles entre fuentes
2. **Coherencia de indicadores:** Comparación AROPE, Gini, S80/S20
3. **Coherencia geográfica:** España en EUROSTAT vs Total en INE
4. **Consistencia de valores:** Verificar que métricas comparables estén en rangos similares

## Salida
- **Reportes JSON/CSV:** `data/validated/logs/`
- **Reporte consolidado:** Resumen de compatibilidad entre fuentes

---

In [1]:
# 0. CONFIGURACIÓN
import sys
import pandas as pd
import pyodbc
from datetime import datetime

# Añadir utils al path
sys.path.append("../../")

from utils.validation_framework import ValidationReport
from utils.config import DB_CONNECTION_STRING

# Conexión a SQL Server
conn = pyodbc.connect(DB_CONNECTION_STRING)

print("=" * 80)
print("MÓDULO DE VALIDACIÓN: INTEGRACIÓN INE ↔ EUROSTAT")
print("=" * 80)
print("✅ Framework de validación cargado")
print(f"⏰ Timestamp: {datetime.now().isoformat()}")

MÓDULO DE VALIDACIÓN: INTEGRACIÓN INE ↔ EUROSTAT
✅ Framework de validación cargado
⏰ Timestamp: 2025-11-20T12:15:17.088063


---

## 1. Cargar Tablas Clave para Integración

---

In [2]:
print("=" * 80)
print("CARGANDO TABLAS CLAVE")
print("=" * 80)

# Crear reporte de integración
report = ValidationReport("INTEGRACION_INE_EUROSTAT")

# Cargar tablas INE clave para comparación con EUROSTAT
# IMPORTANTE: Usar INE_AROPE_Hogar que tiene AROP (no Edad_Sexo que solo tiene AROPE completo)
try:
    df_ine_arope = pd.read_sql("SELECT * FROM INE_AROPE_Hogar", conn)
    print(f"✅ INE_AROPE_Hogar: {len(df_ine_arope)} registros")
    print(f"   Columnas: {list(df_ine_arope.columns)}")
    print(f"   Indicadores disponibles: {df_ine_arope['Indicador'].unique().tolist()}")
except Exception as e:
    print(f"❌ Error cargando INE_AROPE_Hogar: {e}")
    report.add_error(f"No se pudo cargar INE_AROPE_Hogar: {e}")
    df_ine_arope = None

# Cargar tabla EUROSTAT comparable (AROP España)
try:
    df_eurostat_arop = pd.read_sql("SELECT * FROM EUROSTAT_AROP_Espana", conn)
    print(f"✅ EUROSTAT_AROP_Espana: {len(df_eurostat_arop)} registros")
    print(f"   Columnas: {list(df_eurostat_arop.columns)}")
except Exception as e:
    print(f"❌ Error cargando EUROSTAT_AROP_Espana: {e}")
    report.add_error(f"No se pudo cargar EUROSTAT_AROP_Espana: {e}")
    df_eurostat_arop = None

# Cargar tablas para Gini y S80/S20
try:
    df_ine_gini = pd.read_sql("SELECT * FROM INE_Gini_S80S20_CCAA", conn)
    print(f"✅ INE_Gini_S80S20_CCAA: {len(df_ine_gini)} registros")
    print(f"   Columnas: {list(df_ine_gini.columns)}")
except Exception as e:
    print(f"❌ Error cargando INE_Gini_S80S20_CCAA: {e}")
    report.add_error(f"No se pudo cargar INE_Gini_S80S20_CCAA: {e}")
    df_ine_gini = None

try:
    df_eurostat_gini = pd.read_sql("SELECT * FROM EUROSTAT_Gini_Espana", conn)
    print(f"✅ EUROSTAT_Gini_Espana: {len(df_eurostat_gini)} registros")
    print(f"   Columnas: {list(df_eurostat_gini.columns)}")
except Exception as e:
    print(f"❌ Error cargando EUROSTAT_Gini_Espana: {e}")
    report.add_error(f"No se pudo cargar EUROSTAT_Gini_Espana: {e}")
    df_eurostat_gini = None

try:
    df_eurostat_s80s20 = pd.read_sql("SELECT * FROM EUROSTAT_S80S20_Espana", conn)
    print(f"✅ EUROSTAT_S80S20_Espana: {len(df_eurostat_s80s20)} registros")
    print(f"   Columnas: {list(df_eurostat_s80s20.columns)}")
except Exception as e:
    print(f"❌ Error cargando EUROSTAT_S80S20_Espana: {e}")
    report.add_error(f"No se pudo cargar EUROSTAT_S80S20_Espana: {e}")
    df_eurostat_s80s20 = None

print("\n" + "=" * 80)

CARGANDO TABLAS CLAVE
✅ INE_AROPE_Hogar: 308 registros
   Columnas: ['Anio', 'Tipo_Hogar', 'Indicador', 'Valor']
   Indicadores disponibles: ['AROPE', 'AROP', 'Carencia Material Severa', 'Baja Intensidad Laboral']
✅ EUROSTAT_AROP_Espana: 10 registros
   Columnas: ['AROP_%', 'sex', 'sex_label', 'age', 'age_label', 'geo_code', 'geo_name', 'Anio']
✅ INE_Gini_S80S20_CCAA: 340 registros
   Columnas: ['Territorio', 'Anio', 'Gini', 'S80/S20']
✅ EUROSTAT_Gini_Espana: 10 registros
   Columnas: ['Gini', 'age', 'age_label', 'geo_code', 'geo_name', 'Anio']
✅ EUROSTAT_S80S20_Espana: 10 registros
   Columnas: ['S80S20_Ratio', 'age', 'age_label', 'sex', 'sex_label', 'geo_code', 'geo_name', 'Anio']



C:\Users\mario\AppData\Local\Temp\ipykernel_15992\1489637253.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ine_arope = pd.read_sql('SELECT * FROM INE_AROPE_Hogar', conn)
C:\Users\mario\AppData\Local\Temp\ipykernel_15992\1489637253.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_eurostat_arop = pd.read_sql('SELECT * FROM EUROSTAT_AROP_Espana', conn)
C:\Users\mario\AppData\Local\Temp\ipykernel_15992\1489637253.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ine_gini = pd.read_sql('SELECT * FROM INE_G

---

## 2. Validación de Coherencia Temporal

Verificar que los años disponibles en INE y EUROSTAT se solapan.

---

In [3]:
print("=" * 80)
print("VALIDACIÓN DE COHERENCIA TEMPORAL")
print("=" * 80)

if df_ine_arope is not None and df_eurostat_arop is not None:
    # Extraer años únicos de INE
    ine_years = (
        set(df_ine_arope["Anio"].unique()) if "Anio" in df_ine_arope.columns else set()
    )

    # EUROSTAT siempre tiene columna 'Anio' en nuestras tablas
    eurostat_year_col = "Anio"

    if eurostat_year_col in df_eurostat_arop.columns:
        eurostat_years = set(df_eurostat_arop[eurostat_year_col].unique())

        # Calcular solapamiento
        common_years = ine_years.intersection(eurostat_years)
        only_ine = ine_years - eurostat_years
        only_eurostat = eurostat_years - ine_years

        print("\n📊 Análisis temporal:")
        print(f"   INE años: {sorted(ine_years)}")
        print(f"   EUROSTAT años: {sorted(eurostat_years)}")
        print("\n🔗 Solapamiento:")
        print(f"   Años comunes: {sorted(common_years)} ({len(common_years)} años)")
        print(f"   Solo INE: {sorted(only_ine)}")
        print(f"   Solo EUROSTAT: {sorted(only_eurostat)}")

        # Evaluar
        if len(common_years) >= 5:
            print(f"\n✅ Solapamiento temporal adecuado ({len(common_years)} años)")
            report.add_info(f"Solapamiento temporal: {len(common_years)} años comunes")
        elif len(common_years) > 0:
            print(f"\n⚠️  Solapamiento temporal limitado ({len(common_years)} años)")
            report.add_warning(
                f"Solapamiento temporal limitado: solo {len(common_years)} años comunes"
            )
        else:
            print("\n❌ Sin solapamiento temporal entre fuentes")
            report.add_error("No hay años comunes entre INE y EUROSTAT")
    else:
        print("⚠️  No se encontró columna de año en tabla EUROSTAT")
        report.add_warning("No se pudo determinar años en EUROSTAT")
else:
    print("⚠️  No se pueden comparar años: tablas no cargadas")
    report.add_warning("Validación temporal omitida: tablas no disponibles")

print("\n" + "=" * 80)

VALIDACIÓN DE COHERENCIA TEMPORAL

📊 Análisis temporal:
   INE años: [np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
   EUROSTAT años: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]

🔗 Solapamiento:
   Años comunes: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)] (10 años)
   Solo INE: [np.int64(2014)]
   Solo EUROSTAT: []

✅ Solapamiento temporal adecuado (10 años)



---

## 3. Validación de Coherencia de Valores

### 3.1 Validación AROP (Riesgo de Pobreza)

Comparar indicadores similares entre fuentes (ej. AROP España).

---

In [4]:
print("=" * 80)
print("VALIDACIÓN DE COHERENCIA DE VALORES")
print("=" * 80)

if (
    df_ine_arope is not None
    and df_eurostat_arop is not None
    and "common_years" in locals()
    and len(common_years) > 0
):
    print("\n📊 Comparando valores de indicadores AROP (riesgo de pobreza)...")
    print(
        "   ⚠️  NOTA: Se compara AROP (solo riesgo pobreza), NO AROPE (indicador compuesto)"
    )

    # Filtrar INE: Total Nacional, Indicador AROP (no AROPE completo)
    if "Tipo_Hogar" in df_ine_arope.columns and "Indicador" in df_ine_arope.columns:
        df_ine_total = df_ine_arope[
            (df_ine_arope["Tipo_Hogar"] == "Total")
            & (df_ine_arope["Indicador"] == "AROP")  # ← CAMBIADO: Usar AROP no AROPE
            & (df_ine_arope["Anio"].isin(common_years))
        ][["Anio", "Valor"]].rename(columns={"Valor": "INE_AROP"})

        print(
            f"   INE (Tipo_Hogar=Total, Indicador=AROP): {len(df_ine_total)} observaciones"
        )

        if len(df_ine_total) == 0:
            print("   ⚠️  No hay datos AROP en INE. Verificar tabla INE_AROPE_Hogar.")
            report.add_warning("No se encontraron datos AROP en INE_AROPE_Hogar")
            df_ine_total = None
    else:
        print("   ⚠️  Estructura de INE no permite filtrar Total y AROP")
        report.add_error("Estructura INE_AROPE_Hogar no tiene columnas esperadas")
        df_ine_total = None

    # Filtrar EUROSTAT: España Total (geo_code=ES, sex=T, age=TOTAL)
    eurostat_geo_col = "geo_code"
    eurostat_sex_col = "sex"
    eurostat_age_col = "age"

    if (
        eurostat_geo_col in df_eurostat_arop.columns
        and eurostat_year_col in df_eurostat_arop.columns
    ):
        # Filtrar España Total en EUROSTAT (geo_code='ES', sex='T', age='TOTAL')
        df_eurostat_spain = df_eurostat_arop[
            (df_eurostat_arop[eurostat_geo_col] == "ES")
            & (df_eurostat_arop[eurostat_sex_col] == "T")
            & (df_eurostat_arop[eurostat_age_col] == "TOTAL")
            & (df_eurostat_arop[eurostat_year_col].isin(common_years))
        ]

        if len(df_eurostat_spain) > 0:
            # Columna de valor en EUROSTAT: 'AROP_%'
            value_col = "AROP_%"

            if value_col in df_eurostat_spain.columns:
                df_eurostat_spain = df_eurostat_spain[
                    [eurostat_year_col, value_col]
                ].rename(
                    columns={eurostat_year_col: "Anio", value_col: "EUROSTAT_AROP"}
                )
                print(
                    f"   EUROSTAT (España, Total): {len(df_eurostat_spain)} observaciones"
                )

                # Merge y comparar
                if df_ine_total is not None:
                    df_comparison = pd.merge(
                        df_ine_total, df_eurostat_spain, on="Anio", how="inner"
                    )

                    if len(df_comparison) > 0:
                        df_comparison["Diferencia"] = (
                            df_comparison["INE_AROP"] - df_comparison["EUROSTAT_AROP"]
                        )
                        df_comparison["Dif_Pct"] = (
                            df_comparison["Diferencia"] / df_comparison["INE_AROP"]
                        ) * 100

                        print(
                            f"\n📊 Comparación INE (AROP) vs EUROSTAT (AROP) - {len(df_comparison)} años:"
                        )
                        print(df_comparison.to_string(index=False))

                        # Evaluar coherencia - Ahora deberían ser prácticamente iguales
                        max_diff = df_comparison["Dif_Pct"].abs().max()

                        if max_diff < 0.5:
                            print(
                                f"\n✅ Coherencia perfecta: diferencia máxima {max_diff:.2f}%"
                            )
                            report.add_info(
                                f"Coherencia INE-EUROSTAT perfecta: dif. máx. {max_diff:.2f}%"
                            )
                        elif max_diff < 2:
                            print(
                                f"\n✅ Coherencia excelente: diferencia máxima {max_diff:.2f}%"
                            )
                            report.add_info(
                                f"Coherencia INE-EUROSTAT excelente: dif. máx. {max_diff:.2f}%"
                            )
                        elif max_diff < 5:
                            print(
                                f"\n✅ Coherencia buena: diferencia máxima {max_diff:.2f}%"
                            )
                            report.add_info(
                                f"Coherencia INE-EUROSTAT buena: dif. máx. {max_diff:.2f}%"
                            )
                        elif max_diff < 10:
                            print(
                                f"\n⚠️  Coherencia aceptable: diferencia máxima {max_diff:.2f}%"
                            )
                            report.add_warning(
                                f"Diferencia INE-EUROSTAT aceptable: hasta {max_diff:.2f}%"
                            )
                        else:
                            print(
                                f"\n❌ Diferencias significativas: hasta {max_diff:.2f}%"
                            )
                            report.add_error(
                                f"Diferencia excesiva INE-EUROSTAT: {max_diff:.2f}% - Revisar metodología"
                            )
                    else:
                        print("   ⚠️  No hay años comunes con datos comparables")
                        report.add_warning(
                            "No hay datos comparables entre INE y EUROSTAT"
                        )
            else:
                print("   ❌ No se encontró columna 'AROP_%' en EUROSTAT")
                report.add_error(
                    "Columna de valor AROP_% no encontrada en EUROSTAT_AROP_Espana"
                )
        else:
            print("   ⚠️  No se encontraron datos de España Total en EUROSTAT")
            report.add_warning(
                "No se encontraron datos comparables (ES, T, TOTAL) en EUROSTAT"
            )
    else:
        print("   ❌ Estructura EUROSTAT incorrecta: faltan columnas geo_code o Anio")
        report.add_error(
            "Estructura EUROSTAT_AROP_Espana no tiene columnas esperadas (geo_code, Anio)"
        )
else:
    print(
        "⚠️  Validación de valores omitida: tablas no disponibles o sin solapamiento temporal"
    )
    report.add_warning("Validación de valores omitida")

print("\n" + "=" * 80)

VALIDACIÓN DE COHERENCIA DE VALORES

📊 Comparando valores de indicadores AROP (riesgo de pobreza)...
   ⚠️  NOTA: Se compara AROP (solo riesgo pobreza), NO AROPE (indicador compuesto)
   INE (Tipo_Hogar=Total, Indicador=AROP): 10 observaciones
   EUROSTAT (España, Total): 10 observaciones

📊 Comparación INE (AROP) vs EUROSTAT (AROP) - 10 años:
 Anio  INE_AROP  EUROSTAT_AROP  Diferencia  Dif_Pct
 2024      19.7           19.7         0.0      0.0
 2023      20.2           20.2         0.0      0.0
 2022      20.4           20.4         0.0      0.0
 2021      21.7           21.7         0.0      0.0
 2020      21.0           21.0         0.0      0.0
 2019      20.7           20.7         0.0      0.0
 2018      21.5           21.5         0.0      0.0
 2017      21.6           21.6         0.0      0.0
 2016      22.3           22.3         0.0      0.0
 2015      22.1           22.1         0.0      0.0

✅ Coherencia perfecta: diferencia máxima 0.00%



In [5]:
print("=" * 80)
print("VALIDACIÓN DE COHERENCIA: ÍNDICE DE GINI")
print("=" * 80)

if (
    df_ine_gini is not None
    and df_eurostat_gini is not None
    and "common_years" in locals()
    and len(common_years) > 0
):
    print("\n📊 Comparando Índice de Gini...")

    # Filtrar INE: Total Nacional
    if "Territorio" in df_ine_gini.columns:
        df_ine_gini_nacional = df_ine_gini[
            (df_ine_gini["Territorio"] == "Total Nacional")
            & (df_ine_gini["Anio"].isin(common_years))
        ][["Anio", "Gini"]].rename(columns={"Gini": "INE_Gini"})

        print(f"   INE (Total Nacional): {len(df_ine_gini_nacional)} observaciones")
    else:
        print("   ⚠️  Estructura de INE no tiene columna 'Territorio'")
        report.add_warning("No se pudo filtrar Total Nacional en INE_Gini_S80S20_CCAA")
        df_ine_gini_nacional = None

    # Filtrar EUROSTAT: España (geo_code='ES')
    if "geo_code" in df_eurostat_gini.columns:
        df_eurostat_gini_es = df_eurostat_gini[
            (df_eurostat_gini["geo_code"] == "ES")
            & (df_eurostat_gini["Anio"].isin(common_years))
        ][["Anio", "Gini"]].rename(columns={"Gini": "EUROSTAT_Gini"})

        print(f"   EUROSTAT (España): {len(df_eurostat_gini_es)} observaciones")
    else:
        print("   ⚠️  Estructura de EUROSTAT no tiene columna 'geo_code'")
        report.add_warning("No se pudo filtrar España en EUROSTAT_Gini_Espana")
        df_eurostat_gini_es = None

    # Merge y comparar
    if df_ine_gini_nacional is not None and df_eurostat_gini_es is not None:
        df_comparison_gini = pd.merge(
            df_ine_gini_nacional, df_eurostat_gini_es, on="Anio", how="inner"
        )

        if len(df_comparison_gini) > 0:
            df_comparison_gini["Diferencia"] = (
                df_comparison_gini["INE_Gini"] - df_comparison_gini["EUROSTAT_Gini"]
            )
            df_comparison_gini["Dif_Pct"] = (
                df_comparison_gini["Diferencia"] / df_comparison_gini["INE_Gini"]
            ) * 100

            print(
                f"\n📊 Comparación Gini INE vs EUROSTAT - {len(df_comparison_gini)} años:"
            )
            print(df_comparison_gini.to_string(index=False))

            # Evaluar coherencia
            max_diff = df_comparison_gini["Dif_Pct"].abs().max()

            if max_diff < 0.5:
                print(
                    f"\n✅ Coherencia Gini perfecta: diferencia máxima {max_diff:.2f}%"
                )
                report.add_info(f"Coherencia Gini perfecta: dif. máx. {max_diff:.2f}%")
            elif max_diff < 2:
                print(
                    f"\n✅ Coherencia Gini excelente: diferencia máxima {max_diff:.2f}%"
                )
                report.add_info(f"Coherencia Gini excelente: dif. máx. {max_diff:.2f}%")
            elif max_diff < 5:
                print(f"\n✅ Coherencia Gini buena: diferencia máxima {max_diff:.2f}%")
                report.add_warning(
                    f"Coherencia Gini aceptable: dif. máx. {max_diff:.2f}%"
                )
            else:
                print(f"\n⚠️  Diferencias Gini significativas: hasta {max_diff:.2f}%")
                report.add_error(f"Diferencia Gini excesiva: {max_diff:.2f}%")
        else:
            print("   ⚠️  No hay años comunes con datos Gini comparables")
            report.add_warning("No hay datos Gini comparables entre INE y EUROSTAT")
else:
    print("⚠️  Validación Gini omitida: tablas no disponibles")
    report.add_warning("Validación Gini omitida")

print("\n" + "=" * 80)


VALIDACIÓN DE COHERENCIA: ÍNDICE DE GINI

📊 Comparando Índice de Gini...
   INE (Total Nacional): 10 observaciones
   EUROSTAT (España): 10 observaciones

📊 Comparación Gini INE vs EUROSTAT - 10 años:
 Anio  INE_Gini  EUROSTAT_Gini  Diferencia  Dif_Pct
 2015     0.346          0.346         0.0      0.0
 2016     0.345          0.345         0.0      0.0
 2017     0.341          0.341         0.0      0.0
 2018     0.332          0.332         0.0      0.0
 2019     0.330          0.330         0.0      0.0
 2020     0.321          0.321         0.0      0.0
 2021     0.330          0.330         0.0      0.0
 2022     0.320          0.320         0.0      0.0
 2023     0.315          0.315         0.0      0.0
 2024     0.312          0.312         0.0      0.0

✅ Coherencia Gini perfecta: diferencia máxima 0.00%



In [6]:
print("=" * 80)
print("VALIDACIÓN DE COHERENCIA: RATIO S80/S20")
print("=" * 80)

if (
    df_ine_gini is not None
    and df_eurostat_s80s20 is not None
    and "common_years" in locals()
    and len(common_years) > 0
):
    print("\n📊 Comparando Ratio S80/S20...")

    # Filtrar INE: Total Nacional
    if "Territorio" in df_ine_gini.columns and "S80/S20" in df_ine_gini.columns:
        df_ine_s80s20_nacional = df_ine_gini[
            (df_ine_gini["Territorio"] == "Total Nacional")
            & (df_ine_gini["Anio"].isin(common_years))
        ][["Anio", "S80/S20"]].rename(columns={"S80/S20": "INE_S80S20"})

        print(f"   INE (Total Nacional): {len(df_ine_s80s20_nacional)} observaciones")
    else:
        print("   ⚠️  Estructura de INE no tiene columnas 'Territorio' o 'S80/S20'")
        report.add_warning("No se pudo filtrar S80/S20 en INE_Gini_S80S20_CCAA")
        df_ine_s80s20_nacional = None

    # Filtrar EUROSTAT: España Total (geo_code='ES', sex='T', age='TOTAL')
    if (
        "geo_code" in df_eurostat_s80s20.columns
        and "S80S20_Ratio" in df_eurostat_s80s20.columns
    ):
        df_eurostat_s80s20_es = df_eurostat_s80s20[
            (df_eurostat_s80s20["geo_code"] == "ES")
            & (df_eurostat_s80s20["sex"] == "T")
            & (df_eurostat_s80s20["age"] == "TOTAL")
            & (df_eurostat_s80s20["Anio"].isin(common_years))
        ][["Anio", "S80S20_Ratio"]].rename(columns={"S80S20_Ratio": "EUROSTAT_S80S20"})

        print(
            f"   EUROSTAT (España, Total): {len(df_eurostat_s80s20_es)} observaciones"
        )
    else:
        print("   ⚠️  Estructura de EUROSTAT no tiene columnas esperadas")
        report.add_warning("No se pudo filtrar S80/S20 en EUROSTAT_S80S20_Espana")
        df_eurostat_s80s20_es = None

    # Merge y comparar
    if df_ine_s80s20_nacional is not None and df_eurostat_s80s20_es is not None:
        df_comparison_s80s20 = pd.merge(
            df_ine_s80s20_nacional, df_eurostat_s80s20_es, on="Anio", how="inner"
        )

        if len(df_comparison_s80s20) > 0:
            df_comparison_s80s20["Diferencia"] = (
                df_comparison_s80s20["INE_S80S20"]
                - df_comparison_s80s20["EUROSTAT_S80S20"]
            )
            df_comparison_s80s20["Dif_Pct"] = (
                df_comparison_s80s20["Diferencia"] / df_comparison_s80s20["INE_S80S20"]
            ) * 100

            print(
                f"\n📊 Comparación S80/S20 INE vs EUROSTAT - {len(df_comparison_s80s20)} años:"
            )
            print(df_comparison_s80s20.to_string(index=False))

            # Evaluar coherencia
            max_diff = df_comparison_s80s20["Dif_Pct"].abs().max()

            if max_diff < 0.5:
                print(
                    f"\n✅ Coherencia S80/S20 perfecta: diferencia máxima {max_diff:.2f}%"
                )
                report.add_info(
                    f"Coherencia S80/S20 perfecta: dif. máx. {max_diff:.2f}%"
                )
            elif max_diff < 2:
                print(
                    f"\n✅ Coherencia S80/S20 excelente: diferencia máxima {max_diff:.2f}%"
                )
                report.add_info(
                    f"Coherencia S80/S20 excelente: dif. máx. {max_diff:.2f}%"
                )
            elif max_diff < 5:
                print(
                    f"\n✅ Coherencia S80/S20 buena: diferencia máxima {max_diff:.2f}%"
                )
                report.add_warning(
                    f"Coherencia S80/S20 aceptable: dif. máx. {max_diff:.2f}%"
                )
            else:
                print(f"\n⚠️  Diferencias S80/S20 significativas: hasta {max_diff:.2f}%")
                report.add_error(f"Diferencia S80/S20 excesiva: {max_diff:.2f}%")
        else:
            print("   ⚠️  No hay años comunes con datos S80/S20 comparables")
            report.add_warning("No hay datos S80/S20 comparables entre INE y EUROSTAT")
else:
    print("⚠️  Validación S80/S20 omitida: tablas no disponibles")
    report.add_warning("Validación S80/S20 omitida")

print("\n" + "=" * 80)

VALIDACIÓN DE COHERENCIA: RATIO S80/S20

📊 Comparando Ratio S80/S20...
   INE (Total Nacional): 10 observaciones
   EUROSTAT (España, Total): 10 observaciones

📊 Comparación S80/S20 INE vs EUROSTAT - 10 años:
 Anio  INE_S80S20  EUROSTAT_S80S20  Diferencia   Dif_Pct
 2015         6.9             6.87        0.03  0.434783
 2016         6.6             6.60        0.00  0.000000
 2017         6.6             6.59        0.01  0.151515
 2018         6.0             6.03       -0.03 -0.500000
 2019         5.9             5.94       -0.04 -0.677966
 2020         5.8             5.77        0.03  0.517241
 2021         6.2             6.19        0.01  0.161290
 2022         5.6             5.63       -0.03 -0.535714
 2023         5.5             5.50        0.00  0.000000
 2024         5.4             5.39        0.01  0.185185

✅ Coherencia S80/S20 excelente: diferencia máxima 0.68%



---

## 3.3 Validación S80/S20

Comparar ratio S80/S20 entre INE y EUROSTAT.

---

---

## 3.2 Validación Gini

Comparar índice de Gini entre INE y EUROSTAT.

---

---

## 4. Resumen de Integración

---

In [7]:
print("=" * 80)
print("RESUMEN DE VALIDACIÓN DE INTEGRACIÓN")
print("=" * 80)

report.print_report()

# Guardar reporte
print("\n" + "=" * 80)
print("GUARDANDO REPORTE")
print("=" * 80)

json_file = report.save_json()
csv_file = report.save_csv()

print("✅ Reportes de integración guardados:")
print(f"   📄 JSON: {json_file}")
print(f"   📄 CSV: {csv_file}")

print("\n" + "=" * 80)
print("🎯 PRÓXIMOS PASOS")
print("=" * 80)

if report.has_errors():
    print("❌ ATENCIÓN: Se encontraron errores críticos de integración")
    print("   1. Revisar reporte de integración en data/validated/logs/")
    print("   2. Verificar coherencia entre fuentes")
    print("   3. Ajustar reglas de integración si es necesario")
    print("   4. NO proceder al análisis hasta resolver errores")
else:
    print("✅ Validación de integración completada exitosamente")
    print("   1. Todas las fuentes son coherentes y compatibles")
    print("   2. Proceder con análisis exploratorio")
    print("   3. Crear notebooks de análisis en:")
    print("      - notebooks/01_analisis_nacional/")
    print("      - notebooks/02_analisis_regional/")
    print("      - notebooks/03_comparativa_europa/")

print("=" * 80)

RESUMEN DE VALIDACIÓN DE INTEGRACIÓN
REPORTE DE VALIDACIÓN: INTEGRACION_INE_EUROSTAT

[INFO] INFORMACIÓN:
  [INFO] Solapamiento temporal: 10 años comunes
  [INFO] Coherencia INE-EUROSTAT perfecta: dif. máx. 0.00%
  [INFO] Coherencia Gini perfecta: dif. máx. 0.00%
  [INFO] Coherencia S80/S20 excelente: dif. máx. 0.68%

[OK] Sin errores críticos

GUARDANDO REPORTE
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INTEGRACION_INE_EUROSTAT_20251120_121517.json
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INTEGRACION_INE_EUROSTAT_20251120_121517.csv
✅ Reportes de integración guardados:
   📄 JSON: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INTEGRACION_INE_EUROSTAT_20251120_121517.json
   📄 CSV: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\IN

In [8]:
# Cerrar conexión
conn.close()
print("✅ Conexión a SQL Server cerrada")

✅ Conexión a SQL Server cerrada
